#RO Project#

1. Read instances' data from file
2. Initialize the parameters read from file
3. n = number of customers
4. c = matrix of costs
5. ml = max load of the vehicle
6. l = linehaul values vector
7. b = backhaul values vector
8. Initialize the matrix s of savings 
9. for i = 1:n
    1. for j = 1:n
        1. If i == j
            1. s_ij = +inf
        2. else
            1. s_ij = c_i0 + c_0j - c_ij
10. Routes = list of empty routes
11. for i = 1:n
    1. Routes_i = (0,i,0)
12. Order the savings s in a non increasing fashion
13. for each route in Routes:
    1. while this route can be feasibly (respecting the constraints) merged further:
	   1.determine the first saving s_ki or s_jl in the ordered list that can feasibly be used to merge the current route with another route containing either the arc (k, 0) or (0, l). 
       	   2. merge the two routes.



In [1]:
import math
import numpy as np
import sys
import os
import time

In [2]:
def euclidean_distance2D(point1, point2):
    
    x1, y1 = point1
    x2, y2 = point2
    
    return (math.sqrt((x1-x2)**2 + (y1-y2)**2))

In [3]:
def is_route_valid (route_to_check):
    
    is_route_possible = True
    cost_of_possible_route_linehaul = 0
    cost_of_possible_route_backhaul = 0
    i = 0
    
    for node_touched in route_to_check[1:-1]:
        
        if linehaul_vector[node_touched] == 0:#linehaul_vector is 0 if a node is not a linehaul node, so we have to go to the next for
            break
        else:
            cost_of_possible_route_linehaul += linehaul_vector[node_touched]
            i+=1
            
    starting_point = i
    if i < len(route_to_check[1:-1]):
        for node_touched in route_to_check[starting_point+1:-1]:
            
            if backhaul_vector[node_touched] == 0 and i != len(route_to_check): #we are trying to linehaul AFTER we started backhauling... constraint broken:
                is_route_possible = False
                #print(i, backhaul_vector[node_touched], node_touched)
            else:
                cost_of_possible_route_backhaul += backhaul_vector[node_touched]
                i+=1
                
        
    if (cost_of_possible_route_backhaul > capacity or cost_of_possible_route_linehaul > capacity):
        is_route_possible = False
        
    if (cost_of_possible_route_linehaul == 0): # we are only backhauling
        is_route_possible = False
        
        
    return is_route_possible

In [4]:
def is_route_merge_valid(id_route1, id_route2):
    
    """First, try if you can merge it with the end of route1 to end of route2"""
        
    possible_route = list( routes[id_route1][:-1] + routes[id_route2][1:]) #the merge itself is just skipping the left and right
    
    is_right_possible = is_route_valid(possible_route)
    """Now the other possible route."""
        
    possible_route = list( routes[id_route2][:-1] + routes[id_route1][1:]) #the merge itself is just skipping the left and right
    
    is_left_possible = is_route_valid(possible_route)
    
    #print(is_left_possible, is_right_possible)
        
    if not is_left_possible and not is_right_possible:
        return ('')
    elif is_left_possible and is_right_possible:
        return ('lr')
    elif is_left_possible:
        return 'l'
    else:
        return 'r'
        
                

In [5]:
def find_first_merge(id_curr_route, available_merges):

    i = routes[id_curr_route][1]
    j = routes[id_curr_route][-2]
    
    for saving in ordered_savings[n_of_customers+1:]:
        
        
        
        if (saving[1][1] == i or saving[1][0] == j):
            
            
            if (saving[1][1] == i):
                
                                
                for merge in available_merges:
                    
                    if (saving[1][0] == routes[merge[0]][-2] and merge[1] == 'l'):
                        return merge
            
            if (saving[1][0] == j):
                
                
                for merge in available_merges:
                    
                    if (saving[1][1] == routes[merge[0]][1] and merge[1] == 'r'):
                
                        return merge
            
            
    
    
    

In [6]:
def merge_two_routes(id_route1, id_route2, direction_of_merge):

    if id_route1== id_route2:
        return
    
    tmp_list1 = list(routes[id_route1])
    tmp_list2 = list(routes[id_route2])
        
    tmp_list1.pop(0)
    tmp_list1.pop(-1)
    tmp_list2.pop(0)
    tmp_list2.pop(-1)
    
    if direction_of_merge == 'l':
        
        routes[id_route1] = tuple([0]+tmp_list2+tmp_list1+[0])
    
    if direction_of_merge == 'r':
        
        routes[id_route1] = tuple([0]+tmp_list1+tmp_list2+[0])
    
    routes[id_route2] = (-1,-1,-1)




In [7]:
def find_best_relocate():
    
    possible_changes = []
    
    for id_route1, route1 in enumerate(routes):
        
        for id_item_to_move, item_to_move in enumerate(route1[1:-1]): # remember that id is actually +1 if considered in the full route
            
            for id_route2, route2 in enumerate(routes):
                #print('route1:', route1)
                #print('item_to_move:', item_to_move)
                #print('route2:', route2)
                
                if (id_route1 == id_route2):
                    temp_route = list(route1[:id_item_to_move+1] + route1[id_item_to_move+2:])
                else:
                    temp_route = route2
                
                for id_moved, item in enumerate(temp_route[1:]): # just as above, careful with id
                    possible_change = list([0] + list(temp_route[1:id_moved+1]) + [item_to_move] + list(temp_route[id_moved+1:]))
                    possible_change_2 = list(route1[:id_item_to_move+1] + route1[id_item_to_move+2:])
                    if (is_route_valid(possible_change)): #and is_route_valid(possible_change_2)):
                        #print(possible_change)
                        #print(list([0] + list(temp_route[1:id_moved+1]) + [item_to_move] + list(temp_route[id_moved+1:])))
                        #print(id_route1, id_route2, id_item_to_move, id_moved)
                        possible_changes += [(possible_change, item_to_move, id_route1, id_route2, id_item_to_move, id_moved)]
                        
                        
    most_improvement = 0
    best_change = ()

    for id_change, change in enumerate(possible_changes):

        cost_of_old_routes = 0
        cost_of_new_routes = 0
        (possible_change, item_to_move, id_route1, id_route2, id_item_to_move, id_moved) = change
        
        if (id_route1 == id_route2):
            
            for idx, item in enumerate(routes[id_route1][:-1]):
                
                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            new_route_1 = []
            new_route_2 = possible_change

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]
        
        else:
            
        
            for idx, item in enumerate(routes[id_route1][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            for idx, item in enumerate(routes[id_route2][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route2][idx+1]]


            new_route_1 = list(routes[id_route1][:id_item_to_move+1] + routes[id_route1][id_item_to_move+2:])
            new_route_2 = possible_change

            for idx, item in enumerate(new_route_1[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_1[idx+1]]

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]

        #print('cost of old routes:', cost_of_old_routes)
        #print('cost of new routes:', cost_of_new_routes)
        
        if (cost_of_new_routes < cost_of_old_routes):
            #print(cost_of_old_routes - cost_of_new_routes)

            if cost_of_old_routes - cost_of_new_routes > most_improvement:
                most_improvement = cost_of_old_routes - cost_of_new_routes
                best_change = ( most_improvement, (id_route1, new_route_1) , (id_route2, new_route_2) )
                #print(most_improvement)
                
                
    
    #print(most_improvement)
    return best_change
            
        

In [40]:
def find_best_exchange():
    
    possible_changes = []
    
    for id_route1, route1 in enumerate(routes):
        
        for id_item_to_exchange1, item_to_exchange1 in enumerate(route1[1:-1]): # remember that id is actually +1 if considered in the full route
            
            for id_route2, route2 in enumerate(routes):
                
                if(id_route1 > id_route2):
                    continue
                        
                
                for id_item_to_exchange2, item_to_exchange2 in enumerate(route2[1:-1]):
                    
                    possible_change1 = list(route1)
                    possible_change2 = list(route2)
                    
                    if id_route1 == id_route2:
                        possible_change1[id_item_to_exchange1+1], possible_change1[id_item_to_exchange2+1] = possible_change1[id_item_to_exchange2+1], possible_change1[id_item_to_exchange1+1]
                        possible_change2[id_item_to_exchange1+1], possible_change2[id_item_to_exchange2+1] = possible_change2[id_item_to_exchange2+1], possible_change2[id_item_to_exchange1+1]
                        #print((possible_change1, possible_change2))
                    
                    else:
                        possible_change1[id_item_to_exchange1+1] = item_to_exchange2
                        possible_change2[id_item_to_exchange2+1] = item_to_exchange1
                        #print((possible_change1, possible_change2))


                
                    if (is_route_valid(possible_change1) and is_route_valid(possible_change2)): #and is_route_valid (list(route1[:id_item_to_move+1] + route1[id_item_to_move+2:]))):

                        #print((possible_change1, possible_change2))

                        possible_changes += [(possible_change1, possible_change2, item_to_exchange1, item_to_exchange2, id_route1, id_route2, id_item_to_exchange1, id_item_to_exchange2)]

                        
    most_improvement = 0
    best_change = ()

    for id_change, change in enumerate(possible_changes):

        cost_of_old_routes = 0
        cost_of_new_routes = 0
        (possible_change1, possible_change2, item_to_exchange1, item_to_exchange2, id_route1, id_route2, id_item_to_exchange1, id_item_to_exchange2) = change
        
        if (id_route1 == id_route2):
            
            for idx, item in enumerate(routes[id_route1][:-1]):
                
                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            new_route_1 = []
            new_route_2 = possible_change2

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]
        
        else:
            
        
            for idx, item in enumerate(routes[id_route1][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            for idx, item in enumerate(routes[id_route2][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route2][idx+1]]


            new_route_1 = possible_change1
            new_route_2 = possible_change2

            for idx, item in enumerate(new_route_1[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_1[idx+1]]

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]
        #print(new_route_1)
        #print(new_route_2)
        #print('cost of old routes:', cost_of_old_routes)
        #print('cost of new routes:', cost_of_new_routes)
        
        if (cost_of_new_routes < cost_of_old_routes):
            #print(cost_of_old_routes - cost_of_new_routes)

            if cost_of_old_routes - cost_of_new_routes > most_improvement:
                most_improvement = cost_of_old_routes - cost_of_new_routes
                best_change = ( most_improvement, (id_route1, new_route_1) , (id_route2, new_route_2) )
                #print(most_improvement)
                
                
    
    #print(most_improvement)
    return best_change

In [41]:
def get_cost_matrix():

    cost_matrix = []

    x_coords = [int(i.split()[0]) for i in content[3:]]
    y_coords = [int(i.split()[1]) for i in content[3:]]

    for i, (x1, y1) in enumerate(zip(x_coords, y_coords)): #could also enumerate over y_coords, the dimensions are equal

        current_row = []

        for j, (x2, y2) in enumerate(zip(x_coords, y_coords)):

            current_row += [euclidean_distance2D((x1,y1), (x2,y2))]

        cost_matrix += [current_row]
    
    cost_matrix = np.array(cost_matrix)
    return cost_matrix
        

In [42]:
def get_savings_matrix ():
    savings_matrix = np.zeros((n_of_customers + 1, n_of_customers + 1))
    for i in range(n_of_customers + 1):
        for j in range(n_of_customers + 1):
            if (i==j):
                savings_matrix[i][j] = sys.maxsize
            else:
                savings_matrix[i][j] = cost_matrix[i][0] + cost_matrix[0][j] - cost_matrix[i][j]
    return savings_matrix

In [43]:
def compute_total_cost():
    total_cost = 0

    for route in routes:

        if route != (-1, -1, -1):

            for idx, item in enumerate(route[:-1]):

                total_cost+=cost_matrix[item, route[idx+1]]
    return total_cost

In [44]:
#MERGE ROUTES BASED ON THE ALGORITHM CONSTRAINTS
def merge_routes():
    
    number_of_routes = n_of_customers
    
    for id_route1, route in enumerate(routes):

        while True:
            
            if (number_of_routes == n_of_vehicles):
                pass
                #return routes

            if (routes[id_route1] == (-1,-1,-1)):
                break

            found_id = -1
            direction_of_merge = 'l'

            available_merges = []
            for id_route2, route2 in enumerate(routes):

                if (routes[id_route2] == (-1,-1,-1)):
                    continue

                if (id_route1 != id_route2):
                        direction_of_merge = is_route_merge_valid(id_route1, id_route2)
                        if direction_of_merge != "":
                                if direction_of_merge == 'l' or direction_of_merge == 'r':
                                    available_merges += [(id_route2, direction_of_merge)]
                                else:
                                    available_merges += [(id_route2, 'l')]
                                    available_merges += [(id_route2, 'r')]


            if (len(available_merges) == 0):
                break
            #print(available_merges)
            found_id, direction_of_merge = find_first_merge(id_route1, available_merges)
            if routes[found_id] != (-1,-1,-1):
                merge_two_routes(id_route1, found_id, direction_of_merge)
                number_of_routes -= 1
                
                
                
                
    return routes

In [45]:
def find_best_improvement(mode):
    new_cost = 0
    previous_cost = total_cost
    threshold = 5

    while previous_cost - new_cost > threshold:

        possible_change = ()
        if mode == 'relocate':
            possible_change = find_best_relocate()
        else:
            possible_change = find_best_exchange()
        previous_cost = new_cost

        if possible_change != ():

            improvement, (old_route1_index, new_route_1), (old_route2_index, new_route_2) = possible_change

            routes[old_route1_index] = new_route_1
            routes[old_route2_index] = new_route_2

            new_cost = previous_cost - improvement

In [46]:
def fix_less_routes(routes):
    
    best_candidate = (sys.maxsize, -1, -1)
    
    for id_route, route in enumerate(routes):
        
        if len(route) > 3: #this way we can not break any other constraint by removing a vertex from a route
            
            if (len(route) == 4):
                if (linehaul_vector[route[2]] == 0):
                    continue
            
            for id_node, node in enumerate(route[1:-1]):
                
                if linehaul_vector[node] == 0: #candidates are over, we are not at backhauls
                    continue
                
                if cost_matrix[0][node] < best_candidate[0]:
                    best_candidate = (cost_matrix[0][node], id_route, id_node+1)
                    
                    
    routes += [(0, routes[best_candidate[1]][best_candidate[2]], 0)]
    routes[best_candidate[1]] = tuple(list(routes[best_candidate[1]])[:best_candidate[2]] + list(routes[best_candidate[1]])[best_candidate[2]+1:])
    
    return routes
    

In [47]:
def fix_wrong_route(routes, id_route_to_fix):
    
    problem = routes[id_route_to_fix][1]
    
    best_candidate = (sys.maxsize, -1, -1)
    
    for id_route, route in enumerate(routes):
        
        if (id_route != id_route_to_fix):
            
            i = 0
            
            for node in route[1:-1]:
                
                if linehaul_vector[node] == 0:
                    break
                
                i+=1
        
            if is_route_valid(list((0, route[i], problem,  0))):
                if cost_matrix[0][route[i]] < best_candidate[0]:
                    best_candidate = (cost_matrix[0][route[i]], id_route, i)

            
        
    routes[id_route_to_fix] = (0, routes[best_candidate[1]][best_candidate[2]], problem,  0)
    routes[best_candidate[1]] = tuple(list(routes[best_candidate[1]])[:best_candidate[2]] + list(routes[best_candidate[1]])[best_candidate[2]+1:])
    
    return routes

    
    

In [48]:
def fix_high_number():
    n_routes = len(routes)
    invalid_routes = True
    while n_routes > n_of_vehicles or invalid_routes:

        possible_change = ()

        possible_change = find_best_relocate()

        if possible_change != ():

            improvement, (old_route1_index, new_route_1), (old_route2_index, new_route_2) = possible_change

            routes[old_route1_index] = new_route_1
            routes[old_route2_index] = new_route_2
            
            n_routes = len(clean_routes(routes))
            
            for route in routes:
                if (route == (-1,-1,-1) or route == [0,0]):
                    continue
                if not is_route_valid(route):
                    invalid_routes = False
                    break

            invalid_routes = not invalid_routes



In [49]:
def clean_routes(routes):
    #Clean invalid routes
    new_routes = []

    for route in routes:

        if route != (-1, -1, -1):

            new_routes += [route]

    routes = new_routes
    
    new_routes = []
    
    for route in routes:

        if route != [0, 0]:

            new_routes += [tuple(route)]

    routes = new_routes
    return routes

In [50]:
local_search_mode = "exchange"

In [51]:
files = os.listdir("Instances")
files.remove('info.txt')
files.sort()

for filename in files:
    if 'A1' not in filename:
        continue
    with open('Instances/' + filename, 'r') as f:
        
        
        content = f.read()
        #print(content)

    #if filename != 'L1.txt':
        #continue
        
    start = time.time()
    #START OF CONSTRUCTION
    content = content.split('\n')[0:-1] #-1 because the last element is a newline, so we skip it

    n_of_customers = int(content[0])
    n_of_vehicles = int(content[2])
    capacity = int(content[3].split()[3])
    linehaul_vector = [0] + [int(i.split()[2]) for i in content[4:]] #linehaul data is in the fourth column
    backhaul_vector = [0] + [int(i.split()[3]) for i in content[4:]] #and backhaul is in the third

    cost_matrix = get_cost_matrix()
    savings_matrix = get_savings_matrix ()
    routes = []
    for i in range(n_of_customers):
        routes += [(0, i+1, 0)]

    #Order savings
    ordered_savings = []

    for (i, j), el in np.ndenumerate(savings_matrix):
        ordered_savings += [(el, (i, j))]

    ordered_savings.sort(key=lambda x: x[0], reverse=True)
    ordered_i_savings = list(ordered_savings)
    ordered_j_savings = list(ordered_savings)
    ordered_i_savings.sort(key=lambda x: (x[1][0], -x[0]))
    ordered_j_savings.sort(key=lambda x: (x[1][1], -x[0]))

    total_cost = compute_total_cost()

    merge_routes()

    constr_time = time.time() - start
    #END OF CONSTRUCTION   
    
    routes = clean_routes(routes)

    #LOCAL SEARCH
    start = time.time()
    print("Before Local Search:")
    print(routes)
    find_best_improvement(local_search_mode)
    
    total_cost = compute_total_cost()
    
    if(len(routes) > n_of_vehicles):
        fix_high_number()
        routes = clean_routes(routes)
        total_cost = compute_total_cost()
    local_searches_time = time.time() - start
    
    routes = clean_routes(routes)
    print("After Local Search:")
    print(routes)            
        
    #LAST CLEANINGS
    
    done = False
    
    while not done:
    
        for id_route, route in enumerate(routes):
        
            if not is_route_valid(route):
                routes = fix_wrong_route(routes, id_route)
                break
            
            done = True
                
            
    
    if len(routes) < n_of_vehicles:
        times = len(routes)
        
        for i in range(n_of_vehicles - times):
            routes = fix_less_routes(routes)
    
    if len(routes) != n_of_vehicles:
            
        print('Found an instance where we have wrong number of routes, instance n: ' + filename + 'has ' + str(len(routes)) + ' and should have ' + str(n_of_vehicles))
    
    
    #LAST CHECK
    
    for id_route, route in enumerate(routes):
        
        if not is_route_valid(route):
            print(filename + ' failed the check on route number (AFTER LOCAL SEARCHES): ' + str(id_route))

    
    #LOG
    path = 'sequential_routes_'+local_search_mode+"/"
    with open(path + filename, 'w') as f:
        f.write('Total cost ' + str(total_cost))
        f.write('\nTime for construction ' + str(constr_time))
        f.write('\nTime for local search ' + str(local_searches_time))
        for i in range(0,len(routes)):
            route = routes[i]
            f.write('\n\nRoute '+str(i))
            f.write('\n'+str(route))
            cost = 0
            for idx, item in enumerate(route[:-1]):
                cost+=cost_matrix[item, route[idx+1]]
            f.write('\nCost ' + str(cost))

Before Local Search:
[(0, 13, 20, 22, 1, 5, 2, 0), (0, 14, 10, 12, 3, 4, 0), (0, 6, 19, 23, 0), (0, 7, 16, 21, 0), (0, 8, 25, 11, 0), (0, 17, 9, 24, 0), (0, 15, 0), (0, 18, 0)]
After Local Search:
[(0, 13, 20, 22, 1, 5, 2, 0), (0, 14, 10, 12, 3, 4, 0), (0, 6, 19, 23, 0), (0, 7, 16, 21, 0), (0, 11, 25, 8, 0), (0, 17, 9, 24, 0), (0, 15, 0), (0, 18, 0)]


In [20]:
#DEBUG

for route in routes:
    if route == (-1,-1,-1):
        continue
    total_l = 0
    total_b = 0
    for el in route:
        if el == 0:
            continue
        
        total_l += linehaul_vector[el]
        total_b += backhaul_vector[el]
    print(route)
    print(total_l)
    print(total_b)

(0, 150, 64, 144, 117, 149, 93, 75, 67, 145, 122, 106, 137, 86, 53, 68, 85, 88, 1, 14, 30, 5, 35, 49, 36, 20, 50, 13, 47, 3, 7, 48, 43, 33, 39, 28, 19, 0)
8370
8482
(0, 58, 99, 146, 63, 79, 70, 100, 91, 65, 73, 56, 133, 134, 103, 95, 69, 92, 42, 2, 31, 17, 27, 38, 6, 11, 8, 15, 37, 9, 40, 21, 45, 29, 16, 46, 0)
8400
8460
(0, 60, 74, 107, 76, 142, 82, 116, 126, 136, 143, 128, 66, 98, 130, 97, 120, 113, 147, 129, 4, 12, 32, 24, 34, 10, 22, 23, 18, 26, 25, 41, 44, 0)
8347
5897
(0, 51, 72, 78, 114, 123, 131, 61, 118, 77, 121, 148, 71, 90, 0)
8426
0
(0, 125, 141, 52, 138, 112, 94, 101, 115, 108, 109, 54, 80, 81, 57, 0)
8487
0
(0, 139, 140, 84, 124, 110, 127, 132, 55, 135, 111, 59, 102, 96, 83, 0)
8115
0
(0, 62, 119, 89, 104, 105, 0)
3457
0
(0, 87, 0)
498
0


In [21]:
#DEBUG
for route in routes:
    
    
    if route != (-1, -1, -1):
        
        print ('Route:')
        print (route)
        
        for item in route[1:-1]:
            
            if linehaul_vector[item] != 0:
                print('L')
                
            if backhaul_vector[item] != 0:
                print('B')
                

Route:
(0, 150, 64, 144, 117, 149, 93, 75, 67, 145, 122, 106, 137, 86, 53, 68, 85, 88, 1, 14, 30, 5, 35, 49, 36, 20, 50, 13, 47, 3, 7, 48, 43, 33, 39, 28, 19, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
Route:
(0, 58, 99, 146, 63, 79, 70, 100, 91, 65, 73, 56, 133, 134, 103, 95, 69, 92, 42, 2, 31, 17, 27, 38, 6, 11, 8, 15, 37, 9, 40, 21, 45, 29, 16, 46, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
Route:
(0, 60, 74, 107, 76, 142, 82, 116, 126, 136, 143, 128, 66, 98, 130, 97, 120, 113, 147, 129, 4, 12, 32, 24, 34, 10, 22, 23, 18, 26, 25, 41, 44, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
B
B
B
B
B
B
B
B
B
B
Route:
(0, 51, 72, 78, 114, 123, 131, 61, 118, 77, 121, 148, 71, 90, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
Route:
(0, 125, 141, 52, 138, 112, 94, 101, 115, 108, 109, 54, 80, 81, 57, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
Route:
(0, 139, 140, 84, 124, 110, 127, 132, 55, 135, 111, 59, 102, 96, 83, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
Route:
(0, 62

In [22]:
#DEBUG
for route in routes:
    if route == (-1,-1,-1):
        continue
    total_l = 0
    total_b = 0
    for el in route:
        if el == 0:
            continue
        
        total_l += linehaul_vector[el]
        total_b += backhaul_vector[el]
    print(route)
    print(total_l)
    print(total_b)

(0, 150, 64, 144, 117, 149, 93, 75, 67, 145, 122, 106, 137, 86, 53, 68, 85, 88, 1, 14, 30, 5, 35, 49, 36, 20, 50, 13, 47, 3, 7, 48, 43, 33, 39, 28, 19, 0)
8370
8482
(0, 58, 99, 146, 63, 79, 70, 100, 91, 65, 73, 56, 133, 134, 103, 95, 69, 92, 42, 2, 31, 17, 27, 38, 6, 11, 8, 15, 37, 9, 40, 21, 45, 29, 16, 46, 0)
8400
8460
(0, 60, 74, 107, 76, 142, 82, 116, 126, 136, 143, 128, 66, 98, 130, 97, 120, 113, 147, 129, 4, 12, 32, 24, 34, 10, 22, 23, 18, 26, 25, 41, 44, 0)
8347
5897
(0, 51, 72, 78, 114, 123, 131, 61, 118, 77, 121, 148, 71, 90, 0)
8426
0
(0, 125, 141, 52, 138, 112, 94, 101, 115, 108, 109, 54, 80, 81, 57, 0)
8487
0
(0, 139, 140, 84, 124, 110, 127, 132, 55, 135, 111, 59, 102, 96, 83, 0)
8115
0
(0, 62, 119, 89, 104, 105, 0)
3457
0
(0, 87, 0)
498
0


In [23]:
#DEBUG
for route in routes:
    
    
    if route != (-1, -1, -1):
        
        print ('Route:')
        print (route)
        
        for item in route[1:-1]:
            
            if linehaul_vector[item] != 0:
                print('L')
                
            if backhaul_vector[item] != 0:
                print('B')


Route:
(0, 150, 64, 144, 117, 149, 93, 75, 67, 145, 122, 106, 137, 86, 53, 68, 85, 88, 1, 14, 30, 5, 35, 49, 36, 20, 50, 13, 47, 3, 7, 48, 43, 33, 39, 28, 19, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
Route:
(0, 58, 99, 146, 63, 79, 70, 100, 91, 65, 73, 56, 133, 134, 103, 95, 69, 92, 42, 2, 31, 17, 27, 38, 6, 11, 8, 15, 37, 9, 40, 21, 45, 29, 16, 46, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
Route:
(0, 60, 74, 107, 76, 142, 82, 116, 126, 136, 143, 128, 66, 98, 130, 97, 120, 113, 147, 129, 4, 12, 32, 24, 34, 10, 22, 23, 18, 26, 25, 41, 44, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
B
B
B
B
B
B
B
B
B
B
Route:
(0, 51, 72, 78, 114, 123, 131, 61, 118, 77, 121, 148, 71, 90, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
Route:
(0, 125, 141, 52, 138, 112, 94, 101, 115, 108, 109, 54, 80, 81, 57, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
Route:
(0, 139, 140, 84, 124, 110, 127, 132, 55, 135, 111, 59, 102, 96, 83, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
Route:
(0, 62